In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [3]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

### Modification 1

In [11]:
# Calculate profit margin for each item
menu_df['Profit_Margin'] = (menu_df['Price'] - menu_df['Cost']) / menu_df['Price']

In [12]:
menu_df.head()

,Group,Item_Number,Ingredient,Price,Cost,Points Earn,Points Redeem,Profit_Margin
0,Primary,Item_1,Chicken,6.5,2.0,65.0,400.0,0.692308
1,Primary,Item_2,Beef,8.0,2.5,80.0,500.0,0.687500
2,Primary,Item_3,Shrimp,7.5,3.0,75.0,600.0,0.600000
3,Primary,Item_4,Tofu,5.5,1.0,55.0,200.0,0.818182
4,Primary,Item_5,Chickpeas,4.5,0.5,45.0,100.0,0.888889


### Modification 2

In [13]:
transactions_df.head()

,Order_Number,Transaction_Date,Transaction_Time,Store_Number,Customer_ID,Item_Number
0,142292,2024-01-14,00:43:25,STR_7,CUST3706,Item_4
1,23329,2024-01-12,13:54:52,STR_4,CUST7969,Item_1
2,23329,2024-01-12,13:54:52,STR_4,CUST7969,Item_5
3,83297,2024-03-17,03:47:30,STR_2,CUST3789,Item_8
4,83297,2024-03-17,03:47:30,STR_2,CUST3789,Item_1


In [18]:
item_counts = transactions_df['Item_Number'].value_counts()
item_counts_df = item_counts.reset_index()
item_counts_df.columns = ['Item_Number', 'Count_Sold']
item_counts_df.head()

,Item_Number,Count_Sold
0,Item_1,67054
1,Item_2,40257
2,Item_8,39994
3,Item_5,26620
4,Item_6,24068


### Modification 3

In [19]:
item_data = pd.merge(menu_df, item_counts_df, on = 'Item_Number')
item_data.head()

,Group,Item_Number,Ingredient,Price,Cost,Points Earn,Points Redeem,Profit_Margin,Count_Sold
0,Primary,Item_1,Chicken,6.5,2.0,65.0,400.0,0.692308,67054
1,Primary,Item_2,Beef,8.0,2.5,80.0,500.0,0.687500,40257
2,Primary,Item_3,Shrimp,7.5,3.0,75.0,600.0,0.600000,18985
3,Primary,Item_4,Tofu,5.5,1.0,55.0,200.0,0.818182,21224
4,Primary,Item_5,Chickpeas,4.5,0.5,45.0,100.0,0.888889,26620


# 3 Item Popularity vs Profit Margin

In [20]:
# Create bubble chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=item_data['Profit_Margin'],
    y=item_data['Count_Sold'],
    mode='markers',
    marker=dict(
        size=item_data['Price'] * 5,  # Adjust the multiplier for appropriate bubble sizes
        color=item_data['Price'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Price ($)')
    ),
    text=item_data['Item_Number'],
    hovertemplate='<b>%{text}</b><br>Profit Margin: %{x:.2f}<br>Quantity Sold: %{y}<br>Price: $%{marker.color:.2f}<extra></extra>'
))

fig.update_layout(
    title='Item Popularity vs. Profit Margin',
    xaxis_title='Profit Margin',
    yaxis_title='Quantity Sold',
    hovermode='closest'
)

# Add a shapes to divide the chart into quadrants
fig.add_shape(type="line", x0=0.5, y0=0, x1=0.5, y1=1, yref="paper", line=dict(color="Red", width=2, dash="dash"))
fig.add_shape(type="line", x0=0, y0=0.5, x1=1, y1=0.5, xref="paper", line=dict(color="Red", width=2, dash="dash"))

# Add annotations for quadrants
fig.add_annotation(x=0.25, y=0.95, xref="paper", yref="paper", text="Low Margin, High Volume", showarrow=False)
fig.add_annotation(x=0.75, y=0.95, xref="paper", yref="paper", text="High Margin, High Volume", showarrow=False)
fig.add_annotation(x=0.25, y=0.05, xref="paper", yref="paper", text="Low Margin, Low Volume", showarrow=False)
fig.add_annotation(x=0.75, y=0.05, xref="paper", yref="paper", text="High Margin, Low Volume", showarrow=False)

fig.show()